In [1]:
# https://ta-lib.github.io/ta-lib-python/index.html
from talib import abstract
import pandas as pd
import numpy as np
from talib.abstract import *
from quantfreedom._typing import (
    plFrame, plSeries, Union, pdFrame, pdSeries,
)
df = pd.read_csv('30min.csv', index_col='time')

In [3]:
def testing(
    func_name: str,
    df_prices: pdFrame,
    cart_product: bool,
    combos: bool,
    **kwargs,
):
    users_args_list = []
    biggest = 1
    info = Function(func_name).info
    input_names_dict = {}
    in_names_key = ''
    
    for in_names_key, in_names_values in info['input_names'].items():
        filled = False
        if isinstance(in_names_values, list):
            for kwarg_keys, kwarg_values in kwargs.items():
                if in_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, list):
                        raise ValueError(
                            f'{in_names_key} must be a list of strings')

                    if isinstance(kwarg_values, list):
                        if not all(isinstance(x, str) for x in kwarg_values):
                            raise ValueError(
                                f'{in_names_key} your list has to be made up of strings')

                        if len(kwarg_values) == 1:
                            raise ValueError(
                                f'{in_names_key} your list length must be greater than 1')

                        if len(kwarg_values) != len(in_names_values):
                            raise ValueError(
                                f'{in_names_key} your list length must be {len(in_names_values)}')

                    input_names_dict[in_names_key] = kwarg_values
                    filled = True
                    break

            if not filled:
                input_names_dict[in_names_key] = in_names_values

        elif isinstance(in_names_values, str):
            for kwarg_keys, kwarg_values in kwargs.items():
                if in_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, str):
                        raise ValueError(
                            f'{in_names_key} must be a string')

                    input_names_dict[in_names_key] = kwarg_values
                    filled = True
                    break

            if filled:
                input_names_dict[in_names_key] = in_names_values

    if bool(info['parameters']):
        param_dict = {}
        for param_names_key, param_names_values in info['parameters'].items():
            filled = False
            for kwarg_keys, kwarg_values in kwargs.items():
                if param_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, (int, float, list)):
                        raise ValueError(
                            f'{param_names_key} must be a int float or a list of ints or floats')

                    if isinstance(kwarg_values, (int, float)):
                        users_args_list.append(np.asarray([kwarg_values]))

                    elif isinstance(kwarg_values, list):
                        if cart_product == False and combos == False and len(info['parameters']) > 1:
                            raise ValueError(
                                f'{param_names_key} you can\'t have list items as args with indicators that have mutiple params without doing a combo or cart product')
                        if not all(isinstance(x, (int, float)) for x in kwarg_values):
                            raise ValueError(
                                f'{param_names_key} your list has to be filled with ints or floats')
                        if len(kwarg_values) == 1:
                            raise ValueError(
                                f'{param_names_key} your list length must be greater than 1')
                        if combos:
                            if biggest == 1 and len(kwarg_values) > 1:
                                biggest = len(kwarg_values)
                            if biggest != len(kwarg_values):
                                raise ValueError(
                                    f'{param_names_key} when using combos all listed items must be same length')

                        users_args_list.append(np.asarray(kwarg_values))
                    param_dict[param_names_key] = kwarg_values
                    filled = True
                    break

            if not filled:
                param_dict[param_names_key] = param_names_values
                users_args_list.append(np.asarray([param_names_values]))
    else:
        param_dict = ()

    if combos and biggest !=1:
        final_tuple = []
        for x in users_args_list:
            if x.size == 1:
                final_tuple.append(np.broadcast_to(x, biggest))
            else:
                final_tuple.append(x)
        final_tuple = tuple(final_tuple)

    elif cart_product and biggest !=1:
        users_args_tuple = tuple(users_args_list)
        n = 1
        for x in users_args_tuple:
            n *= x.size
        final_tuple = np.empty((n, len(users_args_tuple)))

        for i in range(len(users_args_tuple)):
            m = int(n / users_args_tuple[i].size)
            final_tuple[:n, i] = np.repeat(users_args_tuple[i], m)
            n //= users_args_tuple[i].size

        n = users_args_tuple[-1].size
        for k in range(len(users_args_tuple)-2, -1, -1):
            n *= users_args_tuple[k].size
            m = int(n / users_args_tuple[k].size)
            for j in range(1, users_args_tuple[k].size):
                final_tuple[j*m:(j+1)*m, k+1:] = final_tuple[0:m, k+1:]
                final_tuple= final_tuple.T
    elif biggest ==1 and (cart_product or combos):
        raise ValueError('You have to have a list to use cart product or combos')
    else:
        final_tuple = tuple(users_args_list)
    
    i = 0
    c = 0
    ind_settings_tup = ()
    ind_setings_len = final_tuple[0].size
    final_df_list = []
    for c in range(ind_setings_len):
        for x in final_tuple:
            if type(x[c]) == np.int32:
                ind_settings_tup = ind_settings_tup + (int(x[c]),)
            if type(x[c]) == np.float32:
                ind_settings_tup = ind_settings_tup + (float(x[c]),)
        if in_names_key == 'price':
            temp_df = Function(func_name)(
                    df_prices, *ind_settings_tup, price=input_names_dict[in_names_key]
                    )
            temp_df.columns = pd.MultiIndex.from_product(
                list(ind_settings_tup),
                names=info['parameters'].keys()
            )
            final_df_list.append(temp_df)
        elif in_names_key == 'prices':
            temp_df = Function(func_name)(
                    df_prices, *ind_settings_tup, price=input_names_dict[in_names_key]
                    )
            temp_df.columns = pd.MultiIndex.from_product(
                list(ind_settings_tup),
                names=list(info['parameters'].keys())
            )
            final_df_list.append(temp_df)
        ind_settings_tup = ()
    return final_df_list

dfs = testing(
    func_name='stoch',
    fastk_period=[100,9],
    slowk_period=2,
    slowd_period=3,
    prices=['open', 'open', 'high'],
    cart_product=False,
    combos=True,
    df_prices=df,
)

TypeError: Input must be list-like

In [ ]:
dfs[0]

In [ ]:
dfs[0].columns = pd.MultiIndex.from_product

In [ ]:
pd.concat([dfs[0], dfs[1]], axis=1)

In [ ]:
Function('AVGPRICE').info

In [ ]:
Function('CDLBELTHOLD').info

In [ ]:
Function('stoch')(df, *(9,1,5))

In [ ]:
Function